In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
le = LabelEncoder()

In [3]:
wellbeing_df = pd.read_csv('AllAssessment-updated - Copy.csv')
df_for_le = pd.read_csv('AllAssessment-updated - Copy.csv')

In [4]:
#df_for_le.Status = le.fit_transform(df_for_le.Status)
#df_for_le.Status

In [5]:
input_cols = wellbeing_df.drop(['Gender','Age (Yrs)','Patient Name','NHS ID','Status','Performed By','Performed Date'], 
                               axis = 1).columns

In [6]:
training_input_data = wellbeing_df[input_cols]
target_var = wellbeing_df.Status
#input_data.head()
#training_input_data.columns

In [9]:
le_inputs= training_input_data.apply(le.fit_transform)
le_mapped_values =  {}
for col in list(le_inputs.columns):
    le.fit(training_input_data[col])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    le_mapped_values[col] = le_name_mapping
#le_mapped_values

In [9]:
x_train, x_test, y_train, y_test = train_test_split(le_inputs, target_var, test_size = 0.3)

In [10]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
model.score(x_test, y_test)*100
#model.score(x_train, y_train)*100

85.77817531305904

In [11]:
import pickle
pickle.dump(model, open('final_prediction_dtc.pickle', 'wb'))

In [12]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(x_train, y_train)
model.score(x_test, y_test)*100
#model.score(x_train, y_train)*100

88.37209302325581

In [13]:
pickle.dump(model, open('final_prediction_rfc.pickle', 'wb'))

In [11]:
patient_input = {
	'Breathing today': 'Very Bad',
	'Breathing affecting daily activity': 'Much',
	'Breathing affecting physical activity': 'Very Much',
	'Cough today': 'Worse than usual',
	'Daily Sputum production': 'Up to 15mls (1 tablespoon)',
	'Colour of Sputum': 'Yellowish',
	'How much tired today?': 'Worse than usual',
	'Ankle swelling today?': 'NO',
	'Chest Pain today?': 'YES',
	'Heartburn today?': 'NO',
	'Exacerbation of COPD': 'NO',
	'Receiving treatment for COPD flare-up': 'NO'
}

In [12]:
lst = []
for k,v in patient_input.items():
    lst.append(le_mapped_values[k][v])
#lst

In [13]:
model.predict([lst])

array(['Stable'], dtype=object)

In [14]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(x_train, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(x_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(x_test, y_test)))

Learning rate:  0.05
Accuracy score (training): 0.848
Accuracy score (validation): 0.853
Learning rate:  0.075
Accuracy score (training): 0.859
Accuracy score (validation): 0.856
Learning rate:  0.1
Accuracy score (training): 0.867
Accuracy score (validation): 0.860
Learning rate:  0.25
Accuracy score (training): 0.879
Accuracy score (validation): 0.870
Learning rate:  0.5
Accuracy score (training): 0.886
Accuracy score (validation): 0.871
Learning rate:  0.75
Accuracy score (training): 0.879
Accuracy score (validation): 0.866
Learning rate:  1
Accuracy score (training): 0.843
Accuracy score (validation): 0.818
